In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Hello!
# We will perform Data extraction from twitter and then use that data to analysze the sentiments.
# This project will be in Three parts: 
# 1)Data Extraction
# 2)Data Pre - processing
# 3)Data analysis

# 1) Data Extraction

we will import "Tweepy" to extract data from twitter. Also for twitter data we need a Twitter developer account.

# Packages to be imported 

Tweepy = For twitter data extraction, Pandas = To fill the data in a table, re = To replace strings, datetime = To get the date from user

In [2]:
pip install Tweepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 1.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
#Import packages
import tweepy
import pandas as pd
import re
import datetime
from datetime import datetime

# Authentication

We will use all the keys from our developer account.This step is called authentication.

In [4]:
api_key = "JVWwfHwPBzLoDc2DfFaFL5gmU"
api_key_secret = "AeMHhW4ORfwU1nb8eWJg4PP7P8kULKOUPn37nrWFeH1E2W4pZn"
access_token = "2943199105-fvKLhg03gbnMPTBAfMyiDWo8fm574aVsnAcSRml"
access_token_secret = "Iu7NIQYPaXGURY8aeVVozGyV4QQOXZOYOrROp6Ad3641D"

#Authentication
auth = tweepy.OAuthHandler(api_key,api_key_secret)
auth.set_access_token(access_token,access_token_secret)

api = tweepy.API(auth)

# We will define a function to display the tweet and all the other tweet details

n = Tweet index ith_tweet = The details of tweet (We will define this further)

In [5]:
def display_tweet(n, ith_tweet):
        print()
        print(f"Tweet {n}:")
        print(f"Username:{ith_tweet[0]}")
        print(f"Description:{ith_tweet[1]}")
        print(f"Location:{ith_tweet[2]}")
        print(f"Following Count:{ith_tweet[3]}")
        print(f"Follower Count:{ith_tweet[4]}")
        print(f"Total Tweets:{ith_tweet[5]}")
        print(f"Retweet Count:{ith_tweet[6]}")
        print(f"Tweet Text:{ith_tweet[7]}")
        print(f"Hashtags Used:{ith_tweet[8]}")

 # Data Extraction : We will define a function to extract data, This data will be saved in a dataframe "db"

In [6]:
def scrape(words, date_since, numtweet):
 
        # Creating DataFrame using pandas
        db = pd.DataFrame(columns=['username',
                                   'description',
                                   'location',
                                   'following',
                                   'followers',
                                   'totaltweets',
                                   'retweetcount',
                                   'text',
                                   'hashtags'])
 
        # We are using .Cursor() to search
        # through twitter for the required tweets.
        # The number of tweets can be
        # restricted using .items(number of tweets)
        tweets = tweepy.Cursor(api.search_tweets,
                               words, lang="en",
                               since_id=date_since,
                               tweet_mode='extended').items(numtweet)
 
 
        # .Cursor() returns an iterable object. Each item in
        # the iterator has various attributes
        # that you can access to
        # get information about each tweet
        list_tweets = [tweet for tweet in tweets]
 
        # Counter to maintain Tweet Count
        i = 1
 
        # we will iterate over each tweet in the
        # list for extracting information about each tweet
        for tweet in list_tweets:
                username = tweet.user.screen_name
                description = tweet.user.description
                location = tweet.user.location
                following = tweet.user.friends_count
                followers = tweet.user.followers_count
                totaltweets = tweet.user.statuses_count
                retweetcount = tweet.retweet_count
                hashtags = tweet.entities['hashtags']
 
                # Retweets can be distinguished by
                # a retweeted_status attribute,
                # in case it is an invalid reference,
                # except block will be executed
                try:
                        text = tweet.retweeted_status.full_text
                except AttributeError:
                        text = tweet.full_text
                hashtext = list()
                for j in range(0, len(hashtags)):
                        hashtext.append(hashtags[j]['text'])
 
                # Here we are appending all the
                # extracted information in the DataFrame
                ith_tweet = [username, description,
                             location, following,
                             followers, totaltweets,
                             retweetcount, text, hashtext]
                db.loc[len(db)] = ith_tweet
 
                # Function call to print tweet data on screen
                display_tweet(i, ith_tweet)
                i = i+1
        filename = 'tweets_data.csv'
 
        # we will save our database as a CSV file.
        db.to_csv(filename)

# We will ask user to enter the tweets he wants.

In [7]:
words = input("Enter Twitter HashTag to search for")
print("Enter Date since The Tweets are required in yyyy-mm--dd")
date_since = input()
numtweet = int(input("Enter the no of tweets you want : "))

StdinNotImplementedError: raw_input was called, but this frontend does not support input requests.

# We will perform our task by Running the function

In [ ]:
scrape(words, date_since, numtweet)

In [ ]:
Table = pd.read_csv('tweets_data.csv')
Table.head(5)

# 2) Data Pre processing¶

We can see here that the Text in the table contains (username,url and special char), We will eliminate these char. Also we need only the text for sentimental analysis, We will delete the remaining columns

In [ ]:
Table = Table.drop(['Unnamed: 0','username','description','location','following','followers','totaltweets','retweetcount','hashtags'],axis =1)

In [ ]:
Table.head()

# We will add a index column to the dataframe

In [ ]:
Index = range(1,101)
Table['Index']=Index
Table.set_index('Index',inplace = True)

# Cleaning Data

We will use re library to delete the unwanted text

In [ ]:
def Clean_text(text):
    text = re.sub(r'@[A-Za-z0-9]+', '',text)
    text = re.sub(r'#', '',text)
    text = re.sub(r'RT[\s]+', '',text)
    text = re.sub(r'https?:\/\/\S+', '',text)
    return text

Table['text'] = Table['text'].apply(Clean_text)

In [ ]:
Table.head()

In [ ]:
Table.shape

# 3) Data Analysis (Sentimental Analysis)

After cleaning data we will analysize the text into : Positive,Negative or Neutral For this we use TextBlob library

In [ ]:
from textblob import TextBlob

Creating a funnction to return the sentimenal values and then applying to the table.

In [ ]:
def getsubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def getpolarity(text):
    return TextBlob(text).sentiment.polarity

Table['Subjectivity'] = Table['text'].apply(getsubjectivity)
Table['Polarity'] = Table['text'].apply(getpolarity)

Now we have got Subjectivity and Polarity score, We need to add a column where it shows whether the polarity is negative, positive or neutral.

In [ ]:
def getanalysis(score):
    if score < 0 :
        return "Negative"
    elif score == 0 :
        return "Neutral"
    else :
        return "Positive"

Table['Analysis'] = Table['Polarity'].apply(getanalysis)

In [ ]:
Table.head()

We want to know what are the most common words occuring in Analysis.

In [ ]:
Negative_Table = Table[Table['Analysis'] == "Negative"]

In [ ]:
Positive_Table = Table[Table['Analysis'] == "Positive"]

In [ ]:
Neutral_Table = Table[Table['Analysis'] == "Neutral"]

To Extract most common words from the test we need to use wordcloud library

In [ ]:
from wordcloud import WordCloud
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
allwords = ' '.join([text for text in Table['text']])
wordcloud = WordCloud(width = 500, height = 300, random_state = 21,max_font_size = 119).generate(allwords)

plt.imshow(wordcloud,interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
Negativewords = ' '.join([text for text in Negative_Table['text']])
wordcloud = WordCloud(width = 500, height = 300, random_state = 21,max_font_size = 119).generate(Negativewords)

plt.imshow(wordcloud,interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
Positivewords = ' '.join([text for text in Positive_Table['text']])
wordcloud = WordCloud(width = 500, height = 300, random_state = 21,max_font_size = 119).generate(Positivewords)

plt.imshow(wordcloud,interpolation = 'bilinear')
plt.axis('off')
plt.show()

In [ ]:
Neutralwords = ' '.join([text for text in Neutral_Table['text']])
wordcloud = WordCloud(width = 500, height = 300, random_state = 21,max_font_size = 119).generate(Neutralwords)

plt.imshow(wordcloud,interpolation = 'bilinear')
plt.axis('off')
plt.show()

# We will finally use a Bar graph to show the number of positive, negative and neutral Counts

In [ ]:
Table['Analysis'].value_counts()

plt.title('Sentiment Analysis')
plt.xlabel('Sentiment')
plt.ylabel('Count')

Table['Analysis'].value_counts().plot(kind='bar')
plot.show()
 

# Thank You